In [33]:
import requests

response = requests.get("http://127.0.0.1:8000/get-relevant-data/10029")
result = response.json()

In [34]:
result

{'detail': 'Not Found'}

In [15]:
import pandas as pd

In [16]:
patients_df = pd.read_csv('../mimic-iii-clinical-database-demo-1.4\PATIENTS.csv')


In [17]:
patients_df

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,9467,10006,F,2094-03-05 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,1
1,9472,10011,F,2090-06-05 00:00:00,2126-08-28 00:00:00,2126-08-28 00:00:00,NaN,1
2,9474,10013,F,2038-09-03 00:00:00,2125-10-07 00:00:00,2125-10-07 00:00:00,2125-10-07 00:00:00,1
3,9478,10017,F,2075-09-21 00:00:00,2152-09-12 00:00:00,NaN,2152-09-12 00:00:00,1
4,9479,10019,M,2114-06-20 00:00:00,2163-05-15 00:00:00,2163-05-15 00:00:00,2163-05-15 00:00:00,1
...,...,...,...,...,...,...,...,...
95,31838,44083,M,2057-11-15 00:00:00,2114-02-20 00:00:00,2114-02-20 00:00:00,2114-02-20 00:00:00,1
96,31853,44154,M,1878-05-14 00:00:00,2178-05-15 00:00:00,2178-05-15 00:00:00,2178-05-15 00:00:00,1
97,31867,44212,F,2078-06-16 00:00:00,2124-01-29 00:00:00,NaN,2124-01-29 00:00:00,1
98,31870,44222,M,2107-06-27 00:00:00,2182-08-03 00:00:00,2182-08-03 00:00:00,NaN,1


In [18]:
import os
data = {}
database_folder = '../mimic-iii-clinical-database-demo-1.4'
patients_df = pd.read_csv('../mimic-iii-clinical-database-demo-1.4\PATIENTS.csv')

for subject_id in patients_df['subject_id']:
    data[subject_id] = {}
    for file_name in os.listdir(database_folder):
        # Skip PATIENTS.csv file
        if file_name == 'PATIENTS.csv' or file_name =='CAREGIVERS.csv':
            continue
        if file_name.startswith('D_'):
            continue


        if file_name.endswith('.csv'):
            table_name = file_name.split('.')[0]
            df = pd.read_csv(os.path.join(database_folder, file_name),low_memory=False)
            data[subject_id][table_name] = df[df['subject_id'] == subject_id].to_dict(orient='records')


In [21]:
patients_df['subject_id']

0     10006
1     10011
2     10013
3     10017
4     10019
      ...  
95    44083
96    44154
97    44212
98    44222
99    44228
Name: subject_id, Length: 100, dtype: int64

In [41]:
data[10006].keys()

dict_keys(['ADMISSIONS', 'CALLOUT', 'CHARTEVENTS', 'CPTEVENTS', 'DATETIMEEVENTS', 'DIAGNOSES_ICD', 'DRGCODES', 'ICUSTAYS', 'INPUTEVENTS_CV', 'INPUTEVENTS_MV', 'LABEVENTS', 'MICROBIOLOGYEVENTS', 'OUTPUTEVENTS', 'PRESCRIPTIONS', 'PROCEDUREEVENTS_MV', 'PROCEDURES_ICD', 'SERVICES', 'TRANSFERS'])

In [42]:
list(data[10006].keys())

['ADMISSIONS',
 'CALLOUT',
 'CHARTEVENTS',
 'CPTEVENTS',
 'DATETIMEEVENTS',
 'DIAGNOSES_ICD',
 'DRGCODES',
 'ICUSTAYS',
 'INPUTEVENTS_CV',
 'INPUTEVENTS_MV',
 'LABEVENTS',
 'MICROBIOLOGYEVENTS',
 'OUTPUTEVENTS',
 'PRESCRIPTIONS',
 'PROCEDUREEVENTS_MV',
 'PROCEDURES_ICD',
 'SERVICES',
 'TRANSFERS']

In [1]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, text
import pandas as pd
import os

# Create a SQLAlchemy engine with an in-memory SQLite database
engine = create_engine("sqlite:///C:\\Users\\mtalh\\OneDrive\\Desktop\\ML\\Xavor\\bootcamp\\langserve_medicalmind\\mimic3.db")
engine.connect()

# Graphs

In [3]:
!pip install langchain_openai

   ---------------------------------------- 0.0/798.9 kB ? eta -:--:--
   - ------------------------------------- 30.7/798.9 kB 660.6 kB/s eta 0:00:02
   -- ------------------------------------ 61.4/798.9 kB 825.8 kB/s eta 0:00:01
   --- ----------------------------------- 81.9/798.9 kB 657.6 kB/s eta 0:00:02
   -------- ----------------------------- 174.1/798.9 kB 958.1 kB/s eta 0:00:01
   --------- ---------------------------- 194.6/798.9 kB 985.7 kB/s eta 0:00:01
   --------------- ------------------------ 307.2/798.9 kB 1.1 MB/s eta 0:00:01
   ---------------------- ----------------- 450.6/798.9 kB 1.4 MB/s eta 0:00:01
   ------------------------ --------------- 491.5/798.9 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------  798.7/798.9 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 798.9/798.9 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.6.0
    Uninstalling tiktoken-0.6.0:
      Successfu

In [4]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI

def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [5]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser

members = ["Database retriever", "Researcher"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

llm = ChatOpenAI(model="gpt-3.5-turbo")

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [ ]:
import operator
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import functools

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END


# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str


research_agent = create_agent(llm, [tavily_tool], "You are a web researcher.")
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION. PROCEED WITH CAUTION
code_agent = create_agent(
    llm,
    [python_repl_tool],
    "You may generate safe python code to analyze data and generate charts using matplotlib.",
)
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

workflow = StateGraph(AgentState)
workflow.add_node("Researcher", research_node)
workflow.add_node("Coder", code_node)
workflow.add_node("supervisor", supervisor_chain)